# Observables and Observers
In the observer pattern, an object, named **The Subject**, maintains a set of its dependents, called **Observers**, and notifies them automatically of any state changes, usually by calling one of their methods.

### Observer
The `Observer` interface (type) has one method - `update(..)`

In [18]:
interface Observer<T> {
    fun update(value: T)
}

---
### Observable
The `Observable` class is _responsible_ for registration of `Observer` instances.
It is `abstract` to restrict instantiation of it.
Its `register` and `unregister` methods are final (closed), but `onRegister` and `onUnregister` are protected and open (Open/Close Principle). This is saying "you can't change _how_ it registers observers; you can customise behaviour on registration changes".


Note: It is better to reference/collect the observers as weak references if the language supports it (e.g. Kotlin/Java's `WeakReference`).


In [19]:
abstract class Observable<T> {

    protected val observers = mutableSetOf<Observer<T>>()

    fun register(observer: Observer<T>) {
        observers.add(observer)
        onRegister(observer)
    }

    fun unregister(observer: Observer<T>) {
        onUnregister(observer)
        observers.remove(observer)
    }

    open protected fun onRegister(observer: Observer<T>) = Unit
    open protected fun onUnregister(observer: Observer<T>) = Unit
}

---
### Subjects
The ```Subject``` class _"is an"_ ```Observable```. Its _responsibility_ is to enable its owners/holders to publish values at will (SRP's "new functionality").
The class is open, but its method ```publish(value)``` is closed (Open/Close Principle) - not meant to be modified

In [20]:
open class Subject<T> : Observable<T>() {

    public fun publish(value: T) {
        observers.onEach {
            it.update(value)
        }
    }
}

`BehaviorSubject` adds more functionality - the ability to emit the current value upon registration (useful when the subject is a state, e.g. `isLoggedIn`)

In [21]:
class BehaviorSubject<T>(
    initialValue: T
) : Subject<T>() {

    public var current: T = initialValue
        get() {
            return field
        }
        set(value) {
            if (field != value) {
                field = value
                publish(value)
            }
            (value)
        }

    override protected fun onRegister(observer: Observer<T>) {
        observer.update(current)
    }
}

---
# MVP - The Model

A simple, abstract implementation of _a_ Model with a responsibility to implement the properties and observables of "loading".
Notice that there's no way to interact with it other than read/ observe.

In [22]:
abstract class Model {

    interface Properties {
        val isLoading: Boolean
    }

    interface Observables {
        val isLoading: Observable<Boolean>
    }

    protected val loading = BehaviorSubject<Boolean>(false)

    open val properties: Properties = object : Properties {
        override val isLoading: Boolean
            get() = loading.current
    }

    open val observables: Observables = object : Observables {
        override val isLoading: Observable<Boolean>
            get() = loading
    }
}

## ConsumerAccountModel

A subclass of Model responsible for login state (a boolean and a matching valid consumer profile)

In [36]:
data class ConsumerProfile(
    val firstName: String,
    val lastName: String,
    val avatarUrl: String,
    val email: String,
    val lastLogin:DateTime,
    val uuid: String
    // etc...
)

class ConsumerAccountModel : Model(/* various dependencies like http/grpc client, cookies manager etc */) {

    interface Properties : Model.Properties {
        val isSignedIn: Boolean
        val consumerProfile: ConsumerProfile?
    }

    interface Observables : Model.Observables {
        val isSignedIn: Observable<Boolean>
    }

    // the sign in observable/subject
    private val signedIn = BehaviorSubject<Boolean>(false)

    // a variable(!!) for consumer profile
    private var consumerProfile: ConsumerProfile? = null

    // extended properties and observables
    override val properties: ConsumerAccountModel.Properties = object : ConsumerAccountModel.Properties {

        override val isSignedIn: Boolean = signedIn.current

        override val isLoading: Boolean = super@ConsumerAccountModel.properties.isLoading

        override val consumerProfile: ConsumerProfile?
            get() = this@ConsumerAccountModel.consumerProfile
    }

    override val observables: ConsumerAccountModel.Observables = object : ConsumerAccountModel.Observables {

        override val isSignedIn: Observable<Boolean> = signedIn

        override val isLoading: Observable<Boolean> = super@ConsumerAccountModel.observables.isLoading
    }

    fun signIn(username: String, password: String) {
        consumerProfile = null
        signedIn.current = false

        loading.current = true
        try {
            // actually attempt sign in, e.g.
            consumerProfile = performUsernamePasswordSignIn(username, password)
            // then indicate signed in if succeded
            signedIn.current = consumerProfile != null

        } catch (e: Exception) {
            consumerProfile = null
            signedIn.current = false
        } finally {
            loading.current = false
        }
    }

    private fun performUsernamePasswordSignIn(username: String, password: String): ConsumerProfile? {
        var profile: ConsumerProfile? = null
        // login and return profile
        // profile = ...
        return profile
    }
}

It can start by restoring and validating a stored session (if found), or wait till user interaction triggers an action (transacts)

### ConsumerOrdersModel
A Model responsible for maintaining accurate data of the consumer's orders. It auto-loads/unloads when the consumer signs in/out.

In [24]:
import org.joda.time.DateTime

// but first: some handy types:

enum class InstalmentStatus {
    PAID,
    DUE,
    OVERDUE
}

data class Instalment(
    val status: InstalmentStatus,
    val date: DateTime,
    val amount: Float
    // etc.
)

data class Order(
    val id: Long,
    val merchantId: Long,
    val datePlaced: DateTime,
    val amouht: Float,
    val instalments: Set<Instalment>
    // etc.
)

And then the `ConsumerOrdersModel` can be:

In [25]:
import org.joda.time.DateTime

class ConsumerOrdersModel(
    private val accountProperties: ConsumerAccountModel.Properties,
    private val accountObservables: ConsumerAccountModel.Observables
) : Model(/* plus various dependencies like http/grpc clients, etc */) {

    interface Properties : Model.Properties {
        val activeOrders: List<Order>
        val ordersHistory: List<Order>
    }

    interface Observables : Model.Observables {
        val activeOrders: Observable<List<Order>>
        val ordersHistory: Observable<List<Order>>
    }

    private val activeOrders = BehaviorSubject<List<Order>>(listOf())
    private val ordersHistory = BehaviorSubject<List<Order>>(listOf())

    init {

        // if the the consumer is signed in/out - stay up-to-date
        accountObservables.isSignedIn.register(object : Observer<Boolean> {
            override fun update(value: Boolean) {
                if (value) { // i.e signed in
                    refreshAsync()
                } else { // not signed in - no orders
                    activeOrders.current = emptyList()
                    ordersHistory.current = emptyList()
                }
            }
        })

        // is the consumer model is loading - this model should not expose previous values
        accountObservables.isLoading.register(object : Observer<Boolean> {
            override fun update(value: Boolean) {
                if (value) { // i.e. consumer account model is loading (so not signed in)
                    // while loading - can't have any orders exposed to dependants - this would be wrong
                    activeOrders.current = emptyList()
                    ordersHistory.current = emptyList()
                }
            }
        })
    }

    // extended properties and observables
    override val properties: ConsumerOrdersModel.Properties = object : ConsumerOrdersModel.Properties {
        override val activeOrders: List<Order> = this@ConsumerOrdersModel.activeOrders.current
        override val ordersHistory: List<Order> = this@ConsumerOrdersModel.ordersHistory.current
        override val isLoading: Boolean = super@ConsumerOrdersModel.properties.isLoading
    }

    override val observables: ConsumerOrdersModel.Observables = object : ConsumerOrdersModel.Observables {
        override val isLoading: Observable<Boolean> = super@ConsumerOrdersModel.observables.isLoading
        override val activeOrders: Observable<List<Order>> = this@ConsumerOrdersModel.activeOrders
        override val ordersHistory: Observable<List<Order>> = this@ConsumerOrdersModel.ordersHistory
    }

    fun placeOrder(order: Order) {
        loading.current = true
        try {
            performPlaceOrder(order)?.also {
                activeOrders.current = activeOrders.current + it
                ordersHistory.current = ordersHistory.current + it
            }
        } catch (e: Exception) {
        } finally {
            loading.current = false
        }
    }

    private fun performPlaceOrder(order: Order): Order? {
        // place the order - something like client.placeOrder(accountProperties.consumerProfile.uuid, order)
        return order
    }

    /**
     *  fetch consumer's orders and update the orders history and active orders
     */
    private fun refreshAsync() {
        loading.current = true

        val datePlaced = DateTime.now()
        val orders: List<Order> =
        // should be something like ordersClient.getOrders(accountProperties.consumerProfile.uuid)
            // but for now:
            listOf(
                Order(
                    123, 456, datePlaced, 80.0f, setOf(
                        Instalment(InstalmentStatus.DUE, datePlaced.plusDays(0), 20.0f),
                        Instalment(InstalmentStatus.DUE, datePlaced.plusDays(14), 20.0f),
                        Instalment(InstalmentStatus.DUE, datePlaced.plusDays(28), 20.0f),
                        Instalment(InstalmentStatus.DUE, datePlaced.plusDays(42), 20.0f)
                    )
                )
            )
        // correctly update the orders history and active orders
        ordersHistory.current = orders
        activeOrders.current = orders.filter {
            it.instalments.find { instalment ->
                instalment.status != InstalmentStatus.PAID
            } != null
        }

        loading.current = false
    }
}

So the Model as a layer could have 2 instances

In [26]:
val accountModel = ConsumerAccountModel(/* various http clients, cookies and local storage manager etc. */)

val ordersModel = ConsumerOrdersModel(
    accountModel.properties,
    accountModel.observables,
    /* plus various http clients, local storage manager etc. */
)


Notice that:
1. The dependencies are a-cyclic and directed (DAG)
2. `orderModel` **cannot** call sign-in/out; it can observe and respond to changes in, and use the properties of, `accountModel`


---
### ConsumerInstalmentsScheduleModel
Another model in the collection, this one is responsible for an ordered list of all instalments from all orders

In [27]:
class ConsumerInstalmentsScheduleModel(
    private val ordersObservables: ConsumerOrdersModel.Observables,
    private val accountProperties: ConsumerAccountModel.Properties,
) : Model(/* plus various dependencies like http/grpc clients, etc */) {

    interface Properties : Model.Properties {
        val instalments: List<Instalment>
    }

    interface Observables : Model.Observables {
        val instalments: Observable<List<Instalment>>
    }

    private val instalments = BehaviorSubject<List<Instalment>>(listOf())

    init {

        // every time a new orders history list is emitted - update the list of instalments
        ordersObservables.ordersHistory.register(object : Observer<List<Order>> {
            override fun update(value: List<Order>) {
                instalments.current = value.flatMap { it.instalments }.sortedBy { it.date }
            }
        })

        // is the orders model is loading - this model should not expose previous values
        ordersObservables.isLoading.register(object : Observer<Boolean> {
            override fun update(value: Boolean) {
                if (value) { // i.e. orders model is loading (so not exposing orders)
                    // while loading - can't have any instalments exposed to dependants - this would be wrong
                    instalments.current = emptyList()
                }
            }
        })
    }

    fun payInstalment(instalment: Instalment) {
        // IDK - pay it I guess if not PAID, use account model for consumer details
    }
}

And now the Model layer can look like this:

In [28]:
val accountModel = ConsumerAccountModel(/* various http clients, cookies and local storage manager etc. */)

val ordersModel = ConsumerOrdersModel(
    accountModel.properties,
    accountModel.observables,
    /* plus various http clients, local storage manager etc. */
)

val instalmentsModel = ConsumerInstalmentsScheduleModel(
    ordersModel.observables,
    accountModel.properties,
    /* plus various http clients, local storage manager etc. */
)

And once again the Model layer:
1. The dependencies are a-cyclic and directed (DAG)
2. `instalmentsModel` **cannot** call place order; it can observe and respond to changes in, and use the properties of, `ordersModel`

The key is to decide which models depend on which _parts_ of other models - some only need properties and/or observables, other need the whole model.


The MVP's *Presenter* will likely depend on entire/whole instances of Model, so it can manipulate the data.


### Other Models

`DailyInstalmentModel` - installments aggregated by day
`WeeklyInstalmentModel` - installment aggregated by week


In [29]:
import org.joda.time.LocalDate

class DailyInstalmentModel(
    private val ordersModel: ConsumerOrdersModel,
    private val accountProperties: ConsumerAccountModel.Properties,
) {
    interface Properties : Model.Properties {
        val instalments: Map<LocalDate, List<Instalment>>
    }

    interface Observables : Model.Observables {
        val instalments: Observable<Map<LocalDate, List<Instalment>>>
        val focalUpdate: Observable<Pair<LocalDate, List<Instalment>>>
    }
    // ...


}


It depends on the **entire `ConsumerOrdersModel`** in order to allow payment of an instalment for example.

## The View and the View Mediator


#### The View

The view is often a framework of native classes and sdks, e.g. React, Android, iOS, Dart etc.

For the purposes of this demonstration, we'll define a View class and a few related subclasses and interfaces:


In [30]:
abstract class View(val id: Long) {

    private val childViews = mutableListOf<View>()

    /**
     * a handler to aclick event on a visible, enabled view
     */
    interface OnCLickListener {
        fun onClick(v: View)
    }

    var isEnabled: Boolean = false

    var isVisible: Boolean = false

    // add/remove click handlers
    fun addOnClickListener(listener: OnCLickListener) = Unit
    fun removeOnClickListener(listener: OnCLickListener) = Unit

    // find a child view in the hirarchy
    fun findViewById(id: Long): View? {
        if (id == this.id) {
            this
        } else {
            childViews.onEach {
                it.findViewById(id)?.run {
                    return it
                }
            }
        }
        return null
    }

    // and MANY MANY MORE methods to animate, size, layout, position, apply skin/style etc.
}

open class Button(id: Long) : View(id) {
    fun setLabel(text: String?) = Unit
}

class Label(id: Long) : View(id) {
    fun setText(text: CharSequence?) = Unit
}

class ListView(id: Long) : View(id) {

    interface OnItemSelectedListener {
        fun onItemSelected(index: Int, item: Any)
    }

    var items = emptyList<Any?>()

    fun addOnItemSelectedListener(listener: OnItemSelectedListener) = Unit
    fun removeOnItemSelectedListener(listener: OnItemSelectedListener) = Unit
}

class ImageView(id: Long) : View(id) {

    fun fromUrl(url: String) = Unit
}


class ToggleButton(id: Long) : Button(id) {

    interface OnToggleStateChangeListener {
        fun onToggleStateChange(isSelected: Boolean)
    }

    fun addOnToggleStateChangeListener(listener: OnToggleStateChangeListener) = Unit
    fun removeOnToggleStateChangeListener(listener: OnToggleStateChangeListener) = Unit

    var isSelected: Boolean = false
}

#### The ViewMediator

An abstraction over the view, obfuscating the internal knowledge of the view hierarchy and composition, and exposing semantic API to manipulate/ interact with the view from the outside (responsibility).

The View Mediator defines the interface for its view's interactions (which interactions will be allowed and exposed externally).


##### Note: Unlike instances in the Model layer, a View Mediator is not meant to be used by multiple presenters.

This is not a hard-set rule; just keep in mind that this would be an exception.

Similarly, an instance of a presenter will only depend on (interact with) one mediator - but again - this isn't set in stone.

The rule of thumb is that Presenter instance and View Mediator maintain a 1:1 relationship in the most common case.


#### The ConsumerLoginViewMediator

A view mediator for consumer account login can govern the UI components involved in changes to login status, e.g.
1. The label of the login button(s)
2. The welcome message
3. The avatar

In [31]:
interface ConsumerLoginInteractions {
    fun onLoginButtonClicked()
    fun onLogoutButtonClicked()
}

companion object LoginIds {
    const val VIEW_ID_LOGIN_BUTTON = 1L
    const val VIEW_ID_WELCOME_PROMPT = 2L
}

data class UserLoginData(
    val : String,
    val avatarUrl: String,
    val lastLoginDate: String
)


class ConsumerLoginViewMediator(private val root: View, private val interactions: ConsumerLoginInteractions) {
    private val loginButton: Button
    private val welcomePrompt: Label

    init {

        welcomePrompt = root.findViewById(LoginIds.VIEW_ID_WELCOME_PROMPT)?.let {
            require(it is Label)
            it
        } ?: throw RuntimeException("Failed to find welcome label in $root")

        loginButton = root.findViewById(LoginIds.VIEW_ID_LOGIN_BUTTON)?.let {
            require(it is Button)
            it
        } ?: throw RuntimeException("Failed to find login button in $root")
    }


    private val logoutListener = object : View.OnCLickListener {
        override fun onClick(v: View) {
            interactions.onLogoutButtonClicked()
        }
    }

    private val loginListener = object : View.OnCLickListener {
        override fun onClick(v: View) {
            interactions.onLoginButtonClicked()
        }
    }

    fun setLoggedInState(isLoggedIn: Boolean, userLoginData: UserLoginData?) {
        check((isLoggedIn && userLoginData != null) || (!isLoggedIn && userLoginData == null))

        if (isLoggedIn) {
            with(loginButton) {
                removeOnClickListener(loginListener)
                setLabel("Logout")
                addOnClickListener(logoutListener)
            }
            welcomePrompt.isVisible = true
            welcomePrompt.setText("Welcome ${userLoginData!!.name}")
        } else {
            with(loginButton) {
                removeOnClickListener(logoutListener)
                setLabel("Login")
                addOnClickListener(loginListener)

            }
            welcomePrompt.isVisible = false
        }
    }
}

Line_31.jupyter.kts (59:62 - 66) Unresolved reference: name

##### Notice:
1. The mediator is the only one that has access to the View (hierarchy). DIP - _depend on abstractions, not concretions"_
2. There's a 1:1 relationship between Views (components) and Mediators. (Why is that?)
3. The Mediator above **isn't** brokering/ governing the sign in/up flow - only the login status changes (and the relevant interactions). Respect SRP.

#### The ConsumerOrdersHistoryMediator

Similarly - the view mediator for orders may govern a `ListView` and a `ToggleButton` to display all orders or just the active ones

In [32]:
interface ConsumerOrdersInteractions {

    fun onOrderClicked(orderId: Long) // order is clicked on the list
    fun onPayOrderClicked(orderId: Long) // pay button on the list item is clicked
    fun onFilterActiveOrdersToggled(selected: Boolean) // the filter button is toggled
}

companion object OrdersIds {
    const val VIEW_ID_ORDERS_LIST_ = 1L
    const val VIEW_ID_FILTER_BUTTON = 2L
}

data class OrderViewData(
    val name: String,
    val id: Long,
    val merchant: String,
    val amount: Float, // leaving currency out of it
    val amountDue: Float // remained to be paid
)

class ConsumerOrdersHistoryMediator(private val root: View, private val interactions: ConsumerOrdersInteractions) {
    private val ordersList: ListView
    private val activeOrdersFilter: ToggleButton

    private var consumerOrders: List<OrderViewData> = emptyList()

    init {

        ordersList = root.findViewById(OrdersIds.VIEW_ID_ORDERS_LIST_)?.let {
            require(it is ListView)

            // set the item selected handler and map to the interaction
            it.addOnItemSelectedListener(object : ListView.OnItemSelectedListener {

                override fun onItemSelected(index: Int, item: Any) {
                    if (item is OrderViewData) {
                        interactions.onOrderClicked(item.id)
                    }
                }
            })
            it
        } ?: throw RuntimeException("Failed to find orders list in $root")

        activeOrdersFilter = root.findViewById(OrdersIds.VIEW_ID_FILTER_BUTTON)?.let {
            require(it is ToggleButton)

            // add a handler for state changes (toggles):
            it.addOnToggleStateChangeListener(object : ToggleButton.OnToggleStateChangeListener {

                override fun onToggleStateChange(isSelected: Boolean) {
                    interactions.onFilterActiveOrdersToggled(isSelected)
                }
            })
            it
        } ?: throw RuntimeException("Failed to find login button in $root")
    }


    fun showActiveOrders(filterActiveOrders: Boolean) {
        activeOrdersFilter.isSelected = filterActiveOrders
        populateOrdersList()
    }

    fun setConsumerOrders(orders: List<OrderViewData>) {
        consumerOrders = orders
        populateOrdersList()
    }

    private fun populateOrdersList() {
        ordersList.items = if (activeOrdersFilter.isSelected) {
            consumerOrders.filter { it.amountDue > 0.00f }
        } else {
            consumerOrders
        }
    }
}

### The View State
The View State is a another realm of instances responsible to indicate the state of the view.

For example:
1. Which radio/check button is selected
2. Is a dialog/modal shown or hidden (e.g. login dialog, menus)
3. What texts did the user type in each TextEdit/InputElement
4. What item is selected in the list
5. and more.


The View State can also assist in maintaining other state related values which don't directly map to View components - e.g the current state in a Finite State Machine (FSM) represented in the View (see LoginFlow example).


The View State helps to maintain consistency during internal navigation/ transitions (e.g. if the user selects menu items showing different screens and each of them has stateful elements - they can be restored/ repopulated).

If called for - the View State can be stored and fetched (it would be backed by a dedicated model e.g. `SessionViewStateModel`).

The View State is scoped to a view e.g: orders tab (filter, selected order), login.

The structure of a the View State resembles the model sans the transactional parts - it is a class with observables and properties.

In [33]:
class OrderListViewState {

    interface Properties {
        val isFilterOn: Boolean
    }

    interface Observables {
        val isFilterOn: Observable<Boolean>
    }

    private val filterOnSubject = BehaviorSubject<Boolean>(false)

    val properties: Properties = object : Properties {
        override val isFilterOn: Boolean = filterOnSubject.current
    }

    val observables: Observables = object : Observables {
        override val isFilterOn: Observable<Boolean> = filterOnSubject
    }
}

---
### The Presenter

A Presenter is responsible for the presentation logic of a _portion_ of the app/page.
Presentation Login means **What to display** - the _"what to display" refers to both views and values in them.

A common formula for a Presenter would be:
1. Depend on at least on Model or ViewState, and at least on View Mediator
2. Observe changes in the Models and/or View States
3. On observation update (change) - manipulate the view using the View Mediator

For example: `LoginPresenter`:

In [34]:
class LoginPresenter(
    private val accountObservables: ConsumerAccountModel.Observables,
    private val accountProperties: ConsumerAccountModel.Properties,
    private val loginViewMediator: ConsumerLoginViewMediator
) {
    init {
        accountObservables.isSignedIn.register(
            object : Observer<Boolean> {
                override fun update(value: Boolean) {
                    loginViewMediator.setLoggedInState(
                        accountProperties.isSignedIn,
                        accountProperties.consumerProfile?.let {
                            UserLoginData(
                                it.firstName + " " + it.lastName.first() +".", // e.g "Ron S."
                                it.avatarUrl,
                                it.lastLogin.toLocalDate().toString()
                            )
                         }
                    )
                }

            }
        )
    }
}

Line_34.jupyter.kts (16:36 - 49) Unresolved reference: lastLoginDate